# Practicum adapter pattern
## Doel
- Kennis maken met het adapter pattern

## Inleiding
Deze opgave gaat over het adapter pattern.

Het adapter pattern maakt een klasse compatible met een andere klasse, door om deze klasse een wrapper aan te brengen die een bepaalde interface implementeert.

## Berichten

een berichten app bestaat uit de volgende 3 componenten:
- klasse **Message**, een bericht aan de gebruiker
- klasse **MessageInbox**, een klasse (singleton) waarin de berichten zijn opgeslagen
- klasse **MessageViewer**, een klasse waarmee de berichten worden weergegeven aan een gebruiker

## Class Message
Berichten zijn geïmplementeerd in de volgende immutable klasse:
```Java
public final class Message {

    private final String sender;
    private final LocalDateTime datetime;
    private final String text;

    public Message(String sender, LocalDateTime datetime, String text) {
        this.sender = sender;
        this.datetime = datetime;
        this.text = text;
    }

    public String getSender() {
        return sender;
    }

    public LocalDateTime getDatetime() {
        return datetime;
    }

    public String getText() {
        return text;
    }
    
}
```


## Class MessageInbox 
In deze klasse zijn alle berichten opgeslagen.
```Java
public class MessageInbox {

    private static MessageInbox instance;
    
    private MessageInbox() {};

    public static MessageInbox getInstance() {
        if (instance==null) { // non-thread safe maar dat is ok voor nu
            instance=new MessageInbox();
        }
        return instance;
    }

    public List<Message> getMessages() {
        List<Message> messages = new ArrayList<>();
        messages.add(new Message("joe", LocalDateTime.now(), "Ik ben er over 5 minuten" ));
        messages.add(new Message("Els", LocalDateTime.now(), "Ik heb joe gesproken. Hij staat achter ons idee." ));
        messages.add(new Message("ing", LocalDateTime.now(), "Uw tancode is 81824791" ));
        return messages;
    }

}
```

Deze klasse is een eenvoudige versie van de werkelijkheid. Met de methode *getMessages* worden berichten opgehaald. In plaats daarvan zijn de berichten in dit geval ge-hardcode.

## Class MessageViewer
De klasse **MessageViewer** bevat een eenvoudige weergave voor berichten:
```Java
public class MessageViewer {

    public void view(List<Message> messages) {
        for(Message message : messages) {
            System.out.println("From: "+ message.getSender());
            System.out.println("Date/time: "+ message.getDatetime());
            System.out.println(message.getText());
            System.out.println("====================================================================");
        }
        System.out.println(messages.size()+" messages.");
    }

}
```

## Main
Maak de klassen **Message**, **MessageInbox** en **MessageViewer**.

Maak een klasse **Main** met een main-methode.

Zet in main de volgende code om te testen:
```Java
MessageViewer messageViewer = new MessageViewer();
List<Message> messages = MessageInbox.getInstance().getMessages();
messageViewer.view(messages);
```

Als alles correct is overgenomen, zijn 3 berichten te zien.

## Andere soorten berichten
Er zijn vele soorten berichten: SMS, e-mail, Whatsapp, Telegram, Signal, etc..

De code in deze opgave moet óók whatsapp berichten op kunnen halen en ondersteunen.

Om Whatsapp berichten op te halen is een API-beschikbaar (een verzameling klassen) die de Whatsapp berichten kunnen leveren.

De bestaande klassen **Message**, **MessageInbox** en **MessageViewer** mogen niet gewijzigd worden. De uitbreiding met whatsapp-berichten moet daarom plaatsvinden volgens het open-closed princope: Open voor uitbreiding, gesloten voor wijziging.

Het adapter pattern is geschikt in deze situatie.

## Voorbereiden
Er zijn twee varianten van het adapter pattern: class adapter en object adapter.

In Java is de object adapter het meest geschikt. Dit betekent dat er gebruik wordt gemaakt van compositie.

Om dit mogelijk te maken moet er een kleine verandering worden aangebracht in **Message**: De functionaliteit van message moet opgenomen worden in een interface.

Hernoem **Message** eerst naar **StandardMessage**.

Maak vervolgens de interface **Message**:
```Java
public interface Message {

    String getSender();
    LocalDateTime getDatetime();
    String getText();

}
```

Laat de klasse **StandardMessage** de interface **Message** implementeren.

Zorg ervoor dat de gebruikte datatypes **Message** zijn, en niet **StandardMessage**!

Test de main-methode. Als alles goed is gegaan, is er niets veranderd.

Door het maken van de interface **Message** zijn de berichten geschikt voor het object adapter pattern.

## De Whatsapp-api
Hieronder worden de klasse van de Whatsapp-api gegeven, zonder verdere toelichting. Bestudeer de werking van de klassen. Neem de klassen over en plaats deze in de package *whatsapp*. De klassen mogen niet gewijzigd worden! Een dergelijke situatie komt veel voor, vooral bij gebruik van klassen van derden.

Klasse **WhatsAppMsg**:
```Java
public class WhatsAppMsg {

    private WhatsAppThread whatsAppThread;
    private String content;
    private LocalDate date;
    private LocalTime time;

    public WhatsAppMsg(WhatsAppThread whatsAppThread, String content, LocalDate date, LocalTime time) {
        this.whatsAppThread = whatsAppThread;
        this.content = content;
        this.date = date;
        this.time = time;
    }

    public WhatsAppThread getWhatsAppThread() {
        return whatsAppThread;
    }

    public String getContent() {
        return content;
    }

    public LocalDate getDate() {
        return date;
    }

    public LocalTime getTime() {
        return time;
    }
}
```

Klasse **WhatsAppThread**:
```Java
public class WhatsAppThread {

    private final String from;

    public WhatsAppThread(String from) {
        this.from = from;
    }

    public String from() {
        return from;
    }
}
```

Klasse **WhatsAppReader**:
```Java
public class WhatsAppReader {

    public WhatsAppMsg[] fetchMessages() {
        return new WhatsAppMsg[] {
          new WhatsAppMsg(new WhatsAppThread("06010420"), "Daily standup gaat niet door. Kat van de scrummaster heeft vlooien", LocalDate.now(), LocalTime.now()),
          new WhatsAppMsg(new WhatsAppThread("06995464"), "Het probleem met multithreading is opgelost. We gaan nu het decorator pattern implementeren.", LocalDate.now(), LocalTime.now())
        };
    }
}
```

Ook voor **WhatsAppReader** geldt dat de berichten zijn ge-hardcode. In werkelijkheid worden deze gelezen van een server.

## Combineren standaard berichten en Whatsapp berichten
De volgende code is een poging om standaard berichten en Whatsapp berichten te combineren, voor weergave:
```Java
MessageViewer messageViewer = new MessageViewer();
List<Message> messages = MessageInbox.getInstance().getMessages();

WhatsAppReader whatsAppReader = new WhatsAppReader();
for(WhatsAppMsg whatsAppMsg : whatsAppReader.fetchMessages()) {
    messages.add(whatsAppMsg);
}

messageViewer.view(messages);
```
Plaats deze code in de main-methode en probeer het uit te voeren. Wat is het probleem?
(foutmeldingen op de eerste 2 regels zouden er niet moeten zijn, en kunnen veroorzaakt worden doordat ergens als datatype **StandardMessage** wordt gebruikt in plaats van **Message**)

## Adapter maken
Om de Whatsapp-berichten wel te kunnen toevoegen aan de lijst, zou de klasse **WhatsAppMsg** de interface **Message** moeten implementeren. Deze klasse kan/mag echter niet gewijzigd worden!

De oplossing is een adapter.

De klasse **WhatsAppAdapter** is een adapter voor Whatsapp-berichten.

De klasse **WhatsAppAdapter** heeft de volgende eigenschappen:
- Implementeert de interface **Message**
- Via de constructor wordt een verwijziging naar een **WhatsAppMsg** doorgegeven en opgeslagen in een instantie variabele (composition)
- Bevat implementaties van alle methoden uit interface **Message**

Maak de klasse **WhatsAppAdapter**.

Implementeer alle methoden.

Hints:
- getSender(): In Whatsapp-berichten staat de afzender niet rechtstreeks in het bericht, maar in een instantie van **WhatsAppThread**.
- getDatetime(): In Whatapp-berichten worden tijd en datum afzonderlijk opgeslagen. Maak gebruik van de methode *LocalDateTime.of(LocalDate date, LocalTime time)* om ze te combineren.
- getText(): Deze methode is vrij eenvoudig te implementeren. In Whatsapp-berichten wordt de berichttekst *content* genoemd.


## Adapter gebruiken
De laatste stap is het gebruik maken van de adapter.

Een adapter wordt gemaakt met *new WhatsAppAdapter(whatsAppMsg)*.

De adapter kan toegevoegd worden aan de lijst berichten.

Maak een kleine aanpassing in de main-methode, in de regel *messages.add* om het werkend te krijgen met behulp van de adapter.

Als alles goed is, dan is het resultaat een combinatie van de standaard berichten en Whatsapp-berichten, in totaal 5.

## Adapter pattern
In het object adapter pattern wordt een aantal begrippen gebruikt om bepaalde onderdelen van het pattern aan te duiden: Client, Target, Adaptee en Adapter.

Geef bij elk van deze begrippen aan welke klasse uit deze opgave er bij hoort.